# Introduction

I'm adding this notebook to my PRPT repo as of mid-2022 (20220612, to be exact) in order to see how lessons learned from developing this pipeline in mid-2020 can avail me as I try to stand up a similar pipeline in mid-2022.  We have a mandate at the WA PHL to get *something* in place to analyze HAI/AR samples in-house while we await national guidance on a more streamlined/harmonized pipeline for that purpose that could be employed across the states.

I've been developing such in another repo (for now private, since it's based off another dev's work, and I haven't obtained her permission to publish it), but in pursuing that I came to a similar architecture as employed by this project.  Namely, in the PRPT exercise, I created one long script that pulled in various tools for the pipeline from Docker images maintained elsewhere (mostly, the [StaPH-B Docker builds repo](https://github.com/StaPH-B/docker-builds)) and executes them in order.  The aim for this new-and-improved HAI pipeline is to eventually run it on the Terra platform, and so we *will* need it containerized, and I had originally thought that would require packaging everything into one common Docker image.  But some of the dependencies in turn have enormous databases on which they draw, and sought guidance on whether there are upper limits on what constitutes a decent container.  I found [this page](https://docs.dockstore.org/en/stable/advanced-topics/best-practices/best-practices-secure-fair-workflows.html), and it says:

<blockquote>A good rule of thumb is that each image should have a specific purpose. Avoid installing all of the software you need for an entire analysis in one container, instead use multiple containers.</blockquote>

So the approach ends up being pretty much what I employed for the PRPT exercise two years ago, anyways.  To that end, I thought that it would be useful to take another look at this repo, modernize it a bit, and see how it performs as-is versus the pipeline we're developing.  Several of the tools are similar, but the new pipeline that was developed by Hannah Gray differs in that it has several more tools included, and some custom DBs, and finally that it was all orchestrated in a Bash script, a script was only ever written to work on Hannah's EC2 instance.  Therefore it's *incredibly* particular and not generalizeable, with many paths to tools and resources hard-coded into the script.  As I'm modifying hers, my instinct is to transfer all the critical steps over to a Python script or perhaps eventually a true package.  But given how similar the appropriate strategy appears to be, I thought it would be instructive to take another look at this older repo, and what I learned in designing it to date.

# Modernizing the PRPT Script

Ok, first off, I cloned the old repo to my main EC2 instance, and chosen to open it in VS Code (my original development was done in classic browser-based Jupyter notebook servers).  I've added a VS Code workspace file for configuration, and will want to include `launch.json` files to partially automate testing.

One of the first things that I notice is that the expectation for read inputs is that they will have the "`.zip`" extension and need to be extracted, but the example inputs that I have for adapting Hannah's HAI script are all "`.fastq.gz`", which I believe is the more common format, anyways.  I since discovered that Python's standard library [appears to contain a `gzip` package](https://docs.python.org/3/library/gzip.html) that would be the better way to handle it.  The docs contain an example of how to use it to decompress files, which looks like it should retain the original, compressed file in place, too.

...Never mind, I find that, in truth, I already knew about that package, and invoked it in the script.  It's just that it wasn't used to extract reads, but rather a file downloaded from the NCBI FTP site.  So there's nothing new under the sun, just a package that I had forgotten about in the intervening two years.  I just need to implement the suggested code in the... never mind again?!?  I guess I was looking at the "`unzip_fastqc`" function, which deals not with the raw reads but with the output of FastQC summary of quality...  I guess that the major tools used in this pipeline were always compatible with gzipped read files?  Or else I had extracted them by some step executed in the terminal, and outside the context of my old dev notes.  Ignore this change for now.

## Setting up an Environment

But maybe I'm getting ahead of myself.  I suppose first I need to recognize the dependencies of the original PRPT script, and build a new conda env containing them all.  The beauty of this approach is that there shouldn't be many, as the programs actually used for bioinformatic analysis weren't installed locally, but used via containers, as mentioned, so it should just be a few Python packages not included in the standard library:

In [5]:
with open('prpt_script_1.py', 'r') as f:
    for line in f.readlines():
        if line[:7] == "import ":
            print(line, end='')

import argparse
import datetime
import docker
import gzip
import json
import os
import pandas as pd
import re
import shlex
import shutil
import subprocess
import sys


Looks like just "`docker`" and "`pandas`".  Nice and neat.  I just added "`docker-py`" to my "`jupyter`" conda env, rather than creating a brand new one for this.

My next step was going to be to set up the "`launch.json`" file to try the script as currently configured, but I see instead that one of the first args in the script is a path to a local installation of SNVPhyl.  I guess that's one dependency that didn't exist in containerized form when I was last looking.  There's currently a [containerized version of at least some of the tools](https://github.com/StaPH-B/docker-builds/tree/master/snvphyl-tools/1.8.2) that SNVPhyl relies upon, maintained by StaPH-B.  It looks like they unpacked it from the original script, and say they mean for it to be invoked via Nextflow; I'll have to see if a local clone of the SNVPhyl repo or some part of it will be required to finish the rest...

Well, it's too early to get too caught up in that.  Hannah's script doesn't use SNVPhyl; we can choose later whether to handle it somehow.  For now, stick with a local installation.  I see from my "`PRPT_follow-up_20210416_freeze.html`" version of an old notebook (the original of which is now lost), that I had used the version from [the "`cli`" version of the main SNVPhyl repo](https://github.com/phac-nml/snvphyl-galaxy-cli#snvphyl-cli), and that I had trouble getting it to run, until I found out that the program apparently needs input read files renamed to the form "`{sample}_1.fastq`"...  I'll have to see if that's still the case.

...Hmm, I see that the installation instructions for that repo involve doing a `pip install -r requirements.txt` step, but the only package within is "`bioblend`", so it's also probably compatible with an existing conda env.  Ok, I added bioblend and docker-py to my "`gray`" conda env; run this one from there.  ...I also subsequently found that `boto3` is an optional dependency, if you want to ask the script to upload some results to an S3 bucket.

I'll run `git clone https://github.com/phac-nml/snvphyl-galaxy-cli` from `/data/hai`.

## Testing

Ok, my first test of the script was going well until it hit the `get_ftp_file` func, where it stalled on line 553 with the error:

`Exception has occurred: error_perm
550 /genomes/refseq/bacteria/Citrobacter_sp/: No such file or directory`

I tried accessing the NCBI FTP site directly, and was having issues with Firefox not opening URLs that begin with "`ftp://`", and being redirected to the WinSCP program, which wants login credentials.  Instead, I finally found a page that redirected me to the "NCBI's anonymous FTP site", and the trick is that you use the FTP URL, but preface it with the "`https://`" to indicate that you just want to open it in a browser with the hypertext protocol.  Now I'm trying to open the "`/genomes/refseq/bacteria`" page, but it's hanging up indefinitely on me.  Unsure whether it's just trying to load a **REALLY** huge page, or if something else is going wrong.

Ok, anyways, it finally loaded, and it looks like somethings just going wrong with the logic to parse the species names; there are multiple "Citrobacter_sp" entries, including, well, looks like scores of them.  Probably choosing poor logic for a string split to get the URL, because my original test samples didn't include any "Citrobacter_sp" to work with.  When I relaunched to troubleshoot, I decided to pare down the input sample candidates to just the first sample, but that one wasn't the Citrobacter sample.  So I may have to iteratively troubleshoot.


But one major issue with troubleshooting is that apparently I wrote this script before I got used to using the `logging` module.  So it's really limited in the info on what is happening that gets written to the terminal, and apparently nothing at all gets stored in terms of metadata about how the script ran as a permanent record via a file among the outputs.

And I'll definitely want to add logic to capture which versions of these Docker images are being used, and the versions of the tools within.

...Oops; even the first sample failed later on, when trying to save out the combined Abricate dataframe as an Excel file, because I had missed that `openpyxl` is a dependency for that step.  Man, I've really got to get into the practice of putting all import statements up front at the top of the file, instead of buried inside the functions that specifically need them... oh, I see, there is no call to `import openpyxl` in the script, it's just implicit when saving pandas DataFrames out to `.xlsx`.  I don't really know why I had chosen an Excel output, when the [native outputs from abricate are TSVs](https://github.com/tseemann/abricate#output).  I don't know whether I was just trying to make it easier for end users to open with the right program, or if I added any custom formatting (like preserving column widths or something).

But since I have to restart, and SPAdes was taking like 10min for one sample, I really need to also add logic that checks for intermediate results, such as the existence of a `scaffolds.fasta` output from SPAdes.

## Skipping Assembly

Ok, I tried implementing the ability to at least escape the the SPAdes assembly step, but even that functionality is not easy to implement given the way I had originally written the script.  I think I'll need to refactor more significantly before I'll be able to accommodate such a functionality.

Specifically, I tried adding a "`assemblies_dir`" arg to the `arg_parse` options, then just having the `main` func at the bottom only add the "`spades_cmds`" dict to the list of steps to run in "Part 4" of that func if no assemblies dir has been passed.  But downstream steps expect the presence of the `scaffolds.fasta` files in the output dir.  So at the very least I need to add logic to look for the scaffolds files for each of the chromosome and plasmid SPAdes output dirs for each of the samples.  But honestly, I think at this point that the exact way I wrote the whole script a couple of years ago constitutes a case of spaghetti code, and a more radical refactor is called for.

In [2]:
from pathlib import Path

Path('').resolve()

PosixPath('/data/hai/prpt')

In [4]:
import re

# if not thing:
#     print("There's no such a thing, you fool!")

simple_pattern = re.compile(r"^(.*)[rR][12].*$")
print(simple_pattern)
dir(simple_pattern)
simple_pattern.pattern

re.compile('^(.*)[rR][12].*$')


'^(.*)[rR][12].*$'

In [6]:
import docker

CLIENT = docker.from_env()
images = CLIENT.images.list()
print(len(images))

22


In [13]:
dir(images[0])
images[0].tags
images[0].attrs
images[0].labels['software.version']

'1.8.2'

In [15]:
images[0].attrs.get('RepoTags')
images[0].labels

{'base.image': 'ubuntu:focal',
 'description': 'The SNVPhyl-tools are for a pipeline for identifying Single Nucleotide Variants (SNV) within a collection of microbial genomes and constructing a phylogenetic tree.',
 'dockerfile.version': '1',
 'license': 'https://github.com/phac-nml/snvphyl-tools/blob/master/LICENSE',
 'maintainer': 'Jill Hagey',
 'maintainer.email': 'jvhagey@gmail.com',
 'software': 'SNVPhyl-tools',
 'software.version': '1.8.2',
 'website': 'https://github.com/phac-nml/snvphyl-tools'}

In [16]:
local_images = [
    f"{i.labels.get('software')}:{i.labels.get('software.version')}"
    for i in images
]
print(local_images)

['SNVPhyl-tools:1.8.2', 'SPAdes:3.15.4', 'None:None', 'None:None', 'None:None', 'None:None', 'None:None', 'Kraken2:2.1.2', 'None:None', 'None:None', 'None:None', 'None:None', 'iVar:1.3.1', 'None:None', 'Mash:2.3', 'FASTQC:0.11.9', 'Abricate:1.0.0', 'QUAST:5.0.2', 'iVar:1.2.2', 'MultiQC:1.8', 'Trimmomatic:0.39', 'Trimmomatic:0.39']


In [25]:
local_images

['SNVPhyl-tools:1.8.2',
 'SPAdes:3.15.4',
 'None:None',
 'None:None',
 'None:None',
 'None:None',
 'None:None',
 'Kraken2:2.1.2',
 'None:None',
 'None:None',
 'None:None',
 'None:None',
 'iVar:1.3.1',
 'None:None',
 'Mash:2.3',
 'FASTQC:0.11.9',
 'Abricate:1.0.0',
 'QUAST:5.0.2',
 'iVar:1.2.2',
 'MultiQC:1.8',
 'Trimmomatic:0.39',
 'Trimmomatic:0.39']

In [26]:
print(repr(images[3]))
# images[3].attrs
# CLIENT.images.get('trimmomatic')
CLIENT.containers.list()

<Image: 'python:3.8.12-slim'>


[]

In [10]:
def docker_create(
    image_name: str, volumes: dict, logger: logging.Logger, 
    name=None, tty=True, overwrite=True, **kwargs
    ):
    """Launch a new container image with Docker SDK.
    If overwrite=True and a container with the given name
    already exists; stop and remove it."""
    import docker
    import os
    from functools import partial
    CLIENT = docker.from_env()
    USER = str(os.getuid())
    GID = str(os.getgid())
    # Check that image is present
    images = CLIENT.images.list()
    local_img = CLIENT.images.get(image_name)
    if not local_img:
        repo, version = image_name.split(":")
        image_missing_msg = (
            f"Could not find local installation of image: '{repo}'; "
            f"attempting to download version: '{version}' now."
        )
        logger.info(image_missing_msg)
        CLIENT.images.pull(repo, tag=version)

    create_func = partial(
        CLIENT.containers.create,
        group_add=GID,
        image=image_name,
        name=name,
        tty=tty,
        volumes=volumes,
        user=USER,
        **kwargs,
    )
    try:
        container = create_func()
    except docker.errors.APIError:
        old_container = CLIENT.containers.get(name)
        if overwrite:
            old_container_msg = (
                f"Found running container: {old_container}"
            )
            old_container.stop()
            old_container.remove(v=True, force=True)
            container = create_func()
        else:
            container = old_container

    return container

In [11]:
import logging
# from prpt_script_1 import docker_create

volumes = {
    '/data/hai/prpt_test_1/input_reads_2': 
    {'bind': '/inputs', 'mode': 'ro'},
    'data/hai/prpt_test_1/20220614-0642_PRPT':
    {'bind': '/data', 'mode': 'rw'}
}
logger = logging.Logger('prpt_test_logger')
docker_create('staphb/trimmomatic:latest', volumes, logger)

NullResource: Resource ID was not provided

In [13]:
import docker

CLIENT = docker.from_env()
CLIENT.images.pull('staphb/trimmomatic:latest')

<Image: 'staphb/trimmomatic:latest'>

In [ ]:
import gzip
import shutil
with open('/home/joe/file.txt', 'rb') as f_in:
    with gzip.open('/home/joe/file.txt.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)